<a href="https://colab.research.google.com/github/Erickrus/llm/blob/main/google_adk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart
This quickstart guides you through installing the Agent Development Kit (ADK), setting up a basic agent with multiple tools, and running it locally either in the terminal or in the interactive, browser-based dev UI.

This quickstart assumes a local IDE (VS Code, PyCharm, IntelliJ IDEA, etc.) with Python 3.9+ or Java 17+ and terminal access. This method runs the application entirely on your machine and is recommended for internal development.

https://google.github.io/adk-docs/get-started/quickstart/

## 1. Set up Environment & Install ADK

### Create & Activate Virtual Environment (Recommended):

```shell
# Create
python -m venv .venv
# Activate (each new terminal)
# macOS/Linux: source .venv/bin/activate
# Windows CMD: .venv\Scripts\activate.bat
# Windows PowerShell: .venv\Scripts\Activate.ps1
```

Install ADK:

In [ ]:
!pip3 install -q google-adk

## 2. Create Agent Project

### Project structure

You will need to create the following project structure:

```shell
parent_folder/
    multi_tool_agent/
        __init__.py
        agent.py
        .env
```
Create the folder multi_tool_agent:



In [ ]:
!mkdir multi_tool_agent

**Note for Windows users**

When using ADK on Windows for the next few steps, we recommend creating Python files using File Explorer or an IDE because the following commands (mkdir, echo) typically generate files with null bytes and/or incorrect encoding.

`__init__.py`
Now create an `__init__.py` file in the folder:


In [ ]:
!echo "from . import agent" > multi_tool_agent/__init__.py

Your `__init__.py` should now look like this:

```shell
multi_tool_agent/__init__.py
```
```python
from . import agent
```

agent.py
Create an agent.py file in the same folder:

```shell
touch multi_tool_agent/agent.py
```

Copy and paste the following code into agent.py:

`multi_tool_agent/agent.py`


In [ ]:
#@title multi_tool_agent/agent.py
%%writefile multi_tool_agent/agent.py

import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent

def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city for which to retrieve the weather report.

    Returns:
        dict: status and result or error msg.
    """
    if city.lower() == "new york":
        return {
            "status": "success",
            "report": (
                "The weather in New York is sunny with a temperature of 25 degrees"
                " Celsius (77 degrees Fahrenheit)."
            ),
        }
    else:
        return {
            "status": "error",
            "error_message": f"Weather information for '{city}' is not available.",
        }


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    if city.lower() == "new york":
        tz_identifier = "America/New_York"
    else:
        return {
            "status": "error",
            "error_message": (
                f"Sorry, I don't have timezone information for {city}."
            ),
        }

    tz = ZoneInfo(tz_identifier)
    now = datetime.datetime.now(tz)
    report = (
        f'The current time in {city} is {now.strftime("%Y-%m-%d %H:%M:%S %Z%z")}'
    )
    return {"status": "success", "report": report}


root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description=(
        "Agent to answer questions about the time and weather in a city."
    ),
    instruction=(
        "You are a helpful agent who can answer user questions about the time and weather in a city."
    ),
    tools=[get_weather, get_current_time],
)

Writing multi_tool_agent/agent.py


.env

Create a .env file in the same folder:


In [ ]:
!touch multi_tool_agent/.env

More instructions about this file are described in the next section on Set up the model.

<img width=800px src="https://google.github.io/adk-docs/assets/quickstart-flow-tool.png" />

## 3. Set up the model

Your agent's ability to understand user requests and generate responses is powered by a Large Language Model (LLM). Your agent needs to make secure calls to this external LLM service, which requires authentication credentials. Without valid authentication, the LLM service will deny the agent's requests, and the agent will be unable to function.

**Gemini - Google Cloud Vertex AI**

1. Get an API key from [Google AI Studio](https://aistudio.google.com/apikey).

2. When using Python, open the `.env` file located inside (multi_tool_agent/) and copy-paste the following code.

`multi_tool_agent/.env`

In [ ]:
#@title multi_tool_agent/.env
%%writefile multi_tool_agent/.env
GOOGLE_GENAI_USE_VERTEXAI=FALSE
GOOGLE_API_KEY=PASTE_YOUR_ACTUAL_API_KEY_HERE

Overwriting multi_tool_agent/.env


When using Java, define environment variables:

```shell
export GOOGLE_GENAI_USE_VERTEXAI=FALSE
export GOOGLE_API_KEY=PASTE_YOUR_ACTUAL_API_KEY_HERE
```

3. Replace PASTE_YOUR_ACTUAL_API_KEY_HERE with your actual API KEY.

## 4. Run Your Agent

Using the terminal, navigate to the parent directory of your agent project (e.g. using cd ..):

```shell
parent_folder/      <-- navigate to this directory
    multi_tool_agent/
        __init__.py
        agent.py
        .env
```

There are multiple ways to interact with your agent:

### Dev UI (adk web)



Run the following command to launch the **dev UI**.
```shell
adk web
```

Step 1: Open the URL provided (usually http://localhost:8000 or http://127.0.0.1:8000) directly in your browser.

Step 2. In the top-left corner of the UI, you can select your agent in the dropdown. Select "multi_tool_agent".

Troubleshooting

If you do not see "multi_tool_agent" in the dropdown menu, make sure you are running adk web in the parent folder of your agent folder (i.e. the parent folder of multi_tool_agent).

Step 3. Now you can chat with your agent using the textbox:

<img width=800px src="https://google.github.io/adk-docs/assets/adk-web-dev-ui-chat.png" />

Step 4. By using the Events tab at the left, you can inspect individual function calls, responses and model responses by clicking on the actions:

<img width=800px src="https://google.github.io/adk-docs/assets/adk-web-dev-ui-function-call.png" />

On the Events tab, you can also click the Trace button to see the trace logs for each event that shows the latency of each function calls:

<img width=800px src="https://google.github.io/adk-docs/assets/adk-web-dev-ui-trace.png" />

Step 5. You can also enable your microphone and talk to your agent:

Model support for voice/video streaming

In order to use voice/video streaming in ADK, you will need to use Gemini models that support the Live API. You can find the model ID(s) that supports the Gemini Live API in the documentation:

 - [Google AI Studio: Gemini Live API](https://ai.google.dev/gemini-api/docs/models#live-api)
 - [Vertex AI: Gemini Live API](https://cloud.google.com/vertex-ai/generative-ai/docs/live-api)

You can then replace the model string in root_agent in the agent.py file you created earlier (jump to section). Your code should look something like:

```python
root_agent = Agent(
    name="weather_time_agent",
    model="replace-me-with-model-id", #e.g. gemini-2.0-flash-live-001
    ...
```


### Terminal (adk run)

Run the following command, to chat with your Weather agent.

In [ ]:
!adk run multi_tool_agent

Log setup complete: /tmp/agents_log/agent.20250706_071955.log
To access latest log: tail -F /tmp/agents_log/agent.latest.log
Running agent weather_time_agent, type exit to exit.
[user]: hello
[weather_time_agent]: Hello! How can I help you today? I can provide you with the current weather or time in a specific city.

[user]: what's the weather like today?
[weather_time_agent]: Could you please specify which city you're interested in?

[user]: new york
[weather_time_agent]: The weather in New York is sunny with a temperature of 25 degrees Celsius (77 degrees Fahrenheit).

[user]: 



<img width=800px src="https://google.github.io/adk-docs/assets/adk-run.png" />

To exit, use Cmd/Ctrl+C.



### API Server (adk api_server)

```shell
adk api_server
```
enables you to create a local FastAPI server in a single command, enabling you to test local cURL requests before you deploy your agent.

<img width=800px src="https://google.github.io/adk-docs/assets/adk-api-server.png" />

To learn how to use adk api_server for testing, refer to the [documentation on testing](https://google.github.io/adk-docs/get-started/testing/).



## Misc

https://google.github.io/adk-docs/tools/


If you want to wrap the agent as tool, so you can use:
```python
AgentTool(agent=...)
```

Subagent, you can use:
```python
root_agent = Agent(
    name="...",
    model="gemini-2.0-flash",
    description="...",
    instruction="...",
    tools=[...],
    sub_agents=["..."]
)
```